# cooking the best possible web search result (in terms of relevance and speed)

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## test functions

In [ ]:
from agentic_search.functions.web import get_videos

res = await get_videos("how to cook a pizza step by step")
res


## test chains

In [ ]:
from agentic_search.chains.web import get_route_search_type_chain

res = get_route_search_type_chain().invoke({"query": "what is the weather in Tokyo today?"})
res


In [ ]:
res = get_route_search_type_chain().invoke({"query": "how to cook a pizza step by step"})
res


## test capabilities

### search for current business information

In [3]:
from agentic_search.capabilities.web import get_web_search_results

res = await get_web_search_results("what are the top losers in the US stock market today?")
res

[DEBUG] 
invoking web search agent with messages: [SystemMessage(content='You are a precise research assistant with web search capabilities. Your tasks:\n1. Provide accurate, current information\n2. Synthesize multi-source information concisely\n3. Include citations and maintain objectivity\n\nFocus on authoritative, verifiable sources only.\n\nSkip web search if you are completely certain of information.\nOtherwise, perform targeted searches as needed.\nToday is 2025-01-17.', additional_kwargs={}, response_metadata={}), HumanMessage(content='what are the top losers in the US stock market today?Answer in JSON format: {\n                        "content": "your results as a string",\n                        "metadata": "any additional metadata that was attached to the web search results, if any" | null,\n                        "type": "text" | "video" (if the results are videos)\n                        }', additional_kwargs={}, response_metadata={}, id='82d08607-c5e3-412d-9356-ba756ff

{'results': 'Top Stock Market Losers on January 17, 2025:  \n- Nifty Index:  \n  - Infosys: down 5.86%  \n  - Axis Bank: down 4.52%  \n  - Shriram Finance: down 3.71%  \n  - Kotak Mahindra Bank: down 2.60%  \n  - Mahindra & Mahindra: down 2.12%  \n\n- Sensex:  \n  - Infosys: down 5.77%  \n  - Axis Bank: down 4.71%  \n  - Kotak Mahindra Bank: down 2.58%  \n  - Wipro: down 2.15%  \n  - Mahindra & Mahindra: down 2.11%  \n\n- NSE:  \n  - Kalyan Jewellers India: down 6.99%  \n  - Infosys: down 5.86%  \n  - PB Fintech: down 4.90%  \n  - Axis Bank: down 4.52%  \n  - Vijaya Diagnostic Centre: down 4.31%  \n\n- BSE:  \n  - Sterling & Wilson Renewable Energy: down 9.99%  \n  - Kalyan Jewellers India: down 6.93%  \n  - Hatsun Agro Product: down 5.84%  \n  - Infosys: down 5.77%  \n  - Jindal Worldwide: down 4.97%  \n',
 'metadata': None,
 'type': 'text'}

In [2]:
from agentic_search.capabilities.web import get_web_search_results

res = await get_web_search_results("shadow boxing tutorial")
res

[DEBUG] 
invoking web search agent with messages: [SystemMessage(content='You are a precise research assistant with web search capabilities. Your tasks:\n1. Provide accurate, current information\n2. Synthesize multi-source information concisely\n3. Include citations and maintain objectivity\n\nFocus on authoritative, verifiable sources only.\n\nSkip web search if you are completely certain of information.\nOtherwise, perform targeted searches as needed.\nToday is 2025-01-17.', additional_kwargs={}, response_metadata={}), HumanMessage(content='shadow boxing tutorialAnswer in JSON format: {\n                        "content": "your results as a string",\n                        "metadata": "any additional metadata that was attached to the web search results, if any" | null,\n                        "type": "text" | "video" (if the results are videos)\n                        }', additional_kwargs={}, response_metadata={}, id='0f65d83d-8a7e-4608-bc63-30da1a4a8dd2')]

[DEBUG] 
invoking web

{'results': 'https://www.youtube.com/watch?v=i12A6G_LiJo',
 'metadata': 'How to do Shadow Boxing for Beginners | Why Boxers Shadow Box',
 'type': 'video'}